# Model 2



In [ ]:
!pip install pathway bokeh --quiet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [ ]:
df = pd.read_csv('/content/dataset.csv')
places = df['SystemCodeNumber'].unique()
df

In [4]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True).drop(columns = 'ID')


In [5]:
# df['TrafficConditionNearby'].unique() # low/average/high
# df['VehicleType'].unique() # car,bike,truck,cycle
# df['IsSpecialDay'].unique() 0/1
# queuelenth 0-15

# Label encoding for TrafficConditionNearby
df['TrafficConditionNearby'] = df['TrafficConditionNearby'].map({
    'low': 1,
    'average': 2,
    'high': 3
})

# Label encoding for VehicleType
df['VehicleType'] = df['VehicleType'].map({
    'cycle': 1,
    'bike': 2,
    'car': 3,
    'truck': 4
})

In [6]:
df[["SystemCodeNumber","Timestamp", "Occupancy", "Capacity", "VehicleType",
    "TrafficConditionNearby", "QueueLength", "IsSpecialDay"]].to_csv("parking_stream.csv", index=False)

In [7]:
class ParkingSchema(pw.Schema):
    SystemCodeNumber : str
    Timestamp: str   # Timestamp of the observation (should ideally be in ISO format)
    Occupancy: int   # Number of occupied parking spots
    Capacity: int    # Total parking capacity at the location
    VehicleType: int # Type of vehicle (1=cycle, 2=bike, 3=car, 4=truck)
    TrafficConditionNearby: int # Level of traffic (1=low, 2=average, 3=high)
    QueueLength: int # 0-15
    IsSpecialDay: int # 0/1

In [8]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)


In [9]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)


# Step 2: Making a simple pricing function

In [ ]:
# Define a daily tumbling window over the data stream using Pathway
# This block performs temporal aggregation and computes a dynamic price for each day
import datetime

delta_window = (
    data_with_time
    .windowby(
        pw.this.t,  # Event time column to use for windowing (parsed datetime)
        instance=pw.this.day,  # Logical partitioning key: one instance per calendar day
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),  # Fixed-size daily window
        behavior=pw.temporal.exactly_once_behavior()  # Guarantees exactly-once processing semantics
    )
    .groupby(pw.this.SystemCodeNumber)
    .reduce(
        SystemCodeNumber = pw.reducers.argmax(pw.this.t, pw.this.SystemCodeNumber),
        occupancy = pw.reducers.argmax(pw.this.t, pw.this.Occupancy),
        capacity = pw.reducers.argmax(pw.this.t, pw.this.Capacity),
        t = pw.reducers.argmax(pw.this.t, pw.this.t),
        day = pw.reducers.argmax(pw.this.t, pw.this.day),
        VehicleType = pw.reducers.argmax(pw.this.t, pw.this.VehicleType),
        TrafficConditionNearby = pw.reducers.argmax(pw.this.t, pw.this.TrafficConditionNearby),
        QueueLength = pw.reducers.argmax(pw.this.t, pw.this.QueueLength),
        IsSpecialDay = pw.reducers.argmax(pw.this.t, pw.this.IsSpecialDay),

    )
    .with_columns(
        # alpha = 2.0
        # beta = 1.5
        # gamma = 1.0
        # delta = 3.0
        # epsilon = 0.8
        # baseprice 10
        normalized_demand = (
                                2*(pw.this.occupancy/pw.this.capacity)
                             + 1.5*(pw.this.QueueLength)
                                  -(pw.this.TrafficConditionNearby)
                                +3*(pw.this.IsSpecialDay)
                             + 0.8*(pw.this.VehicleType)
                             )
                             /

                           (
                                  (pw.this.occupancy/pw.this.capacity)
                                +(pw.this.QueueLength)
                                +(pw.this.TrafficConditionNearby)
                                +(pw.this.IsSpecialDay)
                               + (pw.this.VehicleType)
                               )

    )
    .with_columns(

                   price = 10 *( 1 + pw.this.normalized_demand)
    )
)

delta_window

In [ ]:
pw.io.csv.write(delta_window, '/content/output.csv')

In [ ]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

In [ ]:
df_1 = pd.read_csv('/content/output.csv')
df_1['t'] = pd.to_datetime(df_1['t'])
df_1['price'] = df_1['price'].round(2)
df_1

In [ ]:
for place in places:
   mask = df_1['SystemCodeNumber'] == place
   df_2 = df_1[mask]
   data = df_2.copy().reset_index(drop=True)

#Updating prices row-by-row using previous row
   for i in range(1, len(data)):
    prev_price = data.loc[i - 1, 'price']
    prev_demand = data.loc[i - 1, 'normalized_demand']
    demand = data.loc[i, 'normalized_demand']
    if (prev_demand) < (demand):
      alpha = 5
      demand_diff = demand - prev_demand
    else:
      alpha = -5
      demand_diff = prev_demand - demand

    data.loc[i, 'price'] = prev_price + alpha * (demand_diff)

   df_1.loc[mask, 'price'] = data['price'].values


df_1['price'] = df_1['price'].round(2)
df_1

# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

In [ ]:
from bokeh.io import output_file, show, curdoc
from bokeh.plotting import figure, ColumnDataSource
from bokeh.models import Select, CustomJS
from bokeh.layouts import column
from bokeh.models import HoverTool
import pandas as pd

df_1['t_ms'] = pd.to_datetime(df_1['t']).astype('int64') // 10**6
places = df_1['SystemCodeNumber'].unique().tolist()

#Preload full data per place
full_data = {
    place: df_1[df_1['SystemCodeNumber'] == place][['t_ms', 'price']].to_dict('list')
    for place in places
}

#Output HTML file
output_file("animated_price_plot.html", title="Animated Price Plot by Location")
curdoc().theme = 'dark_minimal'

#Empty ColumnDataSource for animation
source = ColumnDataSource(data={'t_ms': [], 'price': []})

p = figure(title="Select a Place", x_axis_type="datetime", width=1000, height=400)

p.add_tools(HoverTool(
      tooltips=[
        ("Time", "@t_ms{%F %T}"),
        ("Price", "@price"),
      ],
      formatters={'@t_ms': 'datetime'},
      mode='vline'
      ))

p.line('t_ms', 'price', source=source)
p.scatter('t_ms', 'price', source=source, fill_color="red", size=5)

#(Select)
select = Select(title="Select Place", value=places[0], options=places)

#JS Callback for Animation
select.js_on_change("value", CustomJS(args=dict(source=source, full_data=full_data, plot=p), code="""
    const selected = this.value;
    const data = full_data[selected];
    const times = data.t_ms;
    const prices = data.price;

    // Reset source
    source.data = {t_ms: [], price: []};
    plot.title.text = selected;

    let i = 0;
    const max = times.length;

    // Clear previous interval if any
    if (window.revealTimer) {
        clearInterval(window.revealTimer);
    }

    window.revealTimer = setInterval(() => {
        if (i >= max) {
            clearInterval(window.revealTimer);
            return;
        }
        source.data.t_ms.push(times[i]);
        source.data.price.push(prices[i]);
        source.change.emit();
        i++;
    }, 300);  // delay per point in ms (adjust for speed)
"""))

output_file("animated_price_plot.html", title="Animated Price Plot by Location")

show(column(select, p))

